# Introduction

Dans ce TP on se chargera de manipuler FastApi en utilisant un modèle de prediction des prix d'un meuble en fonction de certaines caractéristiques

# Processus

Dans cette partie on montre notre procédé pour obtenir notre résultat final

On importe dans un premier temps nos différents libraires

In [1]:
from fastapi import FastAPI , Request, Form, Depends
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.responses import HTMLResponse
import uvicorn
from schemas import Furniture
import pickle
import numpy as np 


Ensuite on crée une nouvelle application FastApi

In [2]:
app = FastAPI()

On precise à notre moteur de template où se trouve nos dossiers où sont stockés nos fichiers html et on en fait de même pour nos fichiers statiques à travers la fonction StaticFiles de FastApi

In [3]:
templates = Jinja2Templates(directory="templates")
app.mount("/static", StaticFiles(directory="static"), name="static")

On charge notre modèle 

In [4]:
model = pickle.load(open('model.pkl','rb'))

/home/lamtoro/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.0 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


On definit ensuite une route avec une méthode GET qui se chargera de retourner notre page index.html

In [5]:
@app.get("/", response_class=HTMLResponse)
async def read_item(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

On définit aussi une route avec une methode POST qui va envoyer à notre serveur les données du formulaire qui contiendront les valeurs de nos caracteristiques

In [6]:
@app.post('/predict')
async def predict(request: Request,form_data: Furniture = Depends(Furniture.as_form)):

    '''
    pour l'affichage sur html
    '''
    prediction =model.predict(
				[
				[
				form_data.category,
				form_data.sellable_online,
				form_data.other_colors,
				form_data.depth,
                form_data.height,
                form_data.width,

				]])
    prediction_text='Furniture prediction price is : $ {}'.format(prediction)
    return templates.TemplateResponse("index.html", {"request": request,"prediction_text":prediction_text })

Pour faire ce travail on a crée un fichier schemas.py qui contient une classe de type Furniture avec nos differents caractéristiques.

Ensuite le formulaire qui sera passé à notre serveur sera transformé en type Furniture et on passera ensuite nos données à notre model pour faire la prédiction . 

Une fois ceci fini on renvoie une page HTML contenant notre prédiction sur le prix

Tout ce qui suffira par la suite c'est de lancer la commande app.py pour lancer notre application

Ci-dessous notre fichier schemas.py

![Schemas](images/schemas.png)

# Resultat

Dans cette partie on se contentera d'afficher les resultats obtenus 

on lance d'abord notre application

In [ ]:
if __name__ == '__main__':
		uvicorn.run("app:app",
		 host="0.0.0.0",
		 port =8080 ,
		 reload = True)	

INFO:     Will watch for changes in these directories: ['/home/lamtoro/Bureau/MLOPS/FastAPI/Furniture Prediction FastAPI']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [40475] using statreload


Une fois cette étape reussie on ouvre notre navigateur à l'adresse precisée ici à savoir http://0.0.0.0:8080

![Lancement Application 2](images/accueil.png)

On remplit notre formulaire et on obtient le resultat de notre prédiction

![Lancement Application](images/resultat.png)

# Conclusion

Ce TP fut une bonne expérience car il nous a permis de prendre en main FastApi avec son moteur de template Jinja2 pour permettre de consommer notre modele de ML sur le Web. 